# PreML_ClassRebalence_FeatureSelection_Prep_20181003_vF
(without Dask)
- Machine_Learning_vB2_20170802
- A cleaner version of Machine_Learning_vB in the same folder
- preceeded by feature creation notebooks

## What dataframes to get out of this notebook:
- Dataframes of:
    - Dataframe with everything
    - Dataframe with only training data (with class rebalancing & no index values that might leak)
    - Dataframe with only training labels (with class rebalancing & no index values that might leak)
    - Dataframe with only test data (with no class rebalancing & no index values that might leak)
    - Dataframe with only test labels (with no class rebalancing & no index values that might leak)
    - Dataframes for each of the 4 above but also with indexes like UWI and DEPT which were kept off
- Order of creation:
    - Split into train/test dataframes
    - Rebalance only training dataset
    - Split into test data, test label, train data, train label
    - Save versions with ONLY index colummns (UWI & Depth) 
    - Take off (UWI & Depth & Others)
    - Save versions with only relevent information
    - Do ML

## Contents
1. [Read In a HDF5 from the previous notebook that creates the features](#readInFirstHDF5)
2. [Add column for train or test based on a split %, like 80%/20%, split based on well UWI](#trainVsTestCol)
3. [Rebalance the classes by throwing out some of the rows away from the pick and duplicating some rows at or near the known pick.](#rebalanceClasses)
4. [Identify which columns to use as training features](#identifyTrainingFeatures)
5. [Identify which columns to use as labels](#identifyLabelCol)
6. [Split single dataframe into 4 for train-features,train-labels,test-features,test-labels](#splitDataframe)
7. [Machine learning using standard XGBoost classifier and not yet Dask](#machineLearningNoDask)
8. [Evaluate the initial results](#ml_evaluation)
9. [Turning row-by-row classification prediction into single well pick depth prediction](#classificationToPick)


In [99]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
# import pdvega
# import vega
import random
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import mean_squared_error


In [100]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.2
0.23.3


In [101]:
%%timeit
import os
env = %env

91.3 µs ± 964 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [102]:
#### Had to change display options to get this to print in full!
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [103]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS"
features_dir = "../createFeatures/"

## If you open this notebook fresh and jump to a point below where a pick file is read in, you still need to load everything above! 

------------

# Reading in the last hdf5 file<a name="readInFirstHDF5"></a>

In [104]:
h5_to_load = 'df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20181003.h5'
h5_key = 'df'
df_all_Col_preSplit = pd.read_hdf(features_dir+h5_to_load, h5_key)

In [105]:
df_all_Col_preSplit.head()

,CALI,COND,DELT,DEPT,DPHI,DT,GR,ILD,ILM,NPHI,PHID,RHOB,SFL,SFLU,SN,SP,UWI,trainOrTest,SitID,lat,lng,TopHelper_HorID,TopTarget_HorID,TopHelper_DEPTH,TopTarget_DEPTH,TopHelper_HorID_Qual,TopTarget_Qual,NN1_topTarget_DEPTH,NN1_TopHelper_DEPTH,NN1_thickness,topTarget_Depth_predBy_NN1thick,diff_Top_Depth_Real_v_predBy_NN1thick,diff_TopTarget_DEPTH_v_rowDEPT,diff_TopHelper_DEPTH_v_rowDEPT,class_DistFrPick_TopTarget,class_DistFrPick_TopHelper,DistFrom_NN1ThickPredTopDepth_toRowDept,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,NPHI_min_5winSize_dirAroundMin,NPHI_min_5winSize_dirAboveMin,NPHI_min_5winSize_dirAroundMax,NPHI_min_5winSize_dirAboveMax,NPHI_min_5winSize_dirAroundMean,NPHI_min_5winSize_dirAboveMean,NPHI_min_5winSize_dirAbovenLarge,NPHI_min_5winSize_dirAroundnLarge,NPHI_min_7winSize_dirAroundMin,NPHI_min_7winSize_dirAboveMin,NPHI_min_7winSize_dirAroundMax,NPHI_min_7winSize_dirAboveMax,NPHI_min_7winSize_dirAroundMean,NPHI_min_7winSize_dirAboveMean,NPHI_min_7winSize_dirAbovenLarge,NPHI_min_7winSize_dirAroundnLarge,NPHI_min_11winSize_dirAroundMin,NPHI_min_11winSize_dirAboveMin,NPHI_min_11winSize_dirAroundMax,NPHI_min_11winSize_dirAboveMax,NPHI_min_11winSize_dirAroundMean,NPHI_min_11winSize_dirAboveMean,NPHI_min_11winSize_dirAbovenLarge,NPHI_min_11winSize_dirAroundnLarge,NPHI_min_21winSize_dirAroundMin,NPHI_min_21winSize_dirAboveMin,NPHI_min_21winSize_dirAroundMax,NPHI_min_21winSize_dirAboveMax,NPHI_min_21winSize_dirAroundMean,NPHI_min_21winSize_dirAboveMean,NPHI_min_21winSize_dirAbovenLarge,NPHI_min_21winSize_dirAroundnLarge,DPHI_min_5winSize_dirAroundMin,DPHI_min_5winSize_dirAboveMin,DPHI_min_5winSize_dirAroundMax,DPHI_min_5winSize_dirAboveMax,DPHI_min_5winSize_dirAroundMean,DPHI_min_5winSize_dirAboveMean,DPHI_min_5winSize_dirAbovenLarge,DPHI_min_5winSize_dirAroundnLarge,DPHI_min_7winSize_dirAroundMin,DPHI_min_7winSize_dirAboveMin,DPHI_min_7winSize_dirAroundMax,DPHI_min_7winSize_dirAboveMax,DPHI_min_7winSize_dirAroundMean,DPHI_min_7win

In [106]:
len(df_all_Col_preSplit.columns)

176

-------------

# Train vs Test Column creation (IGNORED AS DOING THIS BEFORE KNN NOTEBOOK NOW) <a name="trainVsTestCol"></a>

We'll do this based on UWIs, so we don't have any datapoints from train wells in our test datset. This is more like reality than if we'd sample train and test rows randomally from the whole dataframe.

Get all the UWIs

In [107]:
# UWIs = list(df_all_Col_preSplit['UWI'].unique())

Find the number of wells if you want 80%

In [108]:
# numberOfTrainingWells = math.floor(len(UWIs)*0.8)
# numberOfTrainingWells

Randomly select that number of UWIs for training and the ones left for test

In [109]:
# UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [110]:
# UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [111]:
# print("train",len(UWIs_training))
# print("test",len(UWIs_test))

In [112]:
# df_all_Col_preSplit_wTrainTest = df_all_Col_preSplit.copy()

In [113]:
# df_all_Col_preSplit_wTrainTest['trainOrTest'] = np.where(df_all_Col_preSplit_wTrainTest['UWI'].isin(UWIs_training), 'train', 'test')

In [114]:
# df_all_Col_preSplit_wTrainTest.tail()

--------------

## Split into training and test dataframes

In [115]:
#### Create dataframe that is just the train rows
df_all_Col_train_noRebalance = df_all_Col_preSplit.loc[df_all_Col_preSplit['trainOrTest'] == 'train']

In [116]:
#### Create dataframe that is just the test rows
df_all_Col_test = df_all_Col_preSplit.loc[df_all_Col_preSplit['trainOrTest'] == 'test']

------------

# Rebalance class, aka label, populations to deal with lopsided class populations<a name="rebalanceClasses"></a>

#### Because we have a lot more rows far away from the pick than exactly at the pick or close to the pick, we run the risk of being class heavy in some areas. This can result in not enough ability to identify the sparsely populate classes, like right at the pick. 
#### We'll attemp to deal with this problem by throwing out some of the rows far away from the pick and duplicating some of the rows right at or near the pick.

### THIS SHOULD ONLY BE DONE TO THE TRAINING DATA NOT THE TEST DATA OR THAT IS CHEATING!

In [117]:
#### create a copy for the test below to avoid rewriting accidentally
# df_test_5 = df_all_Col_preSplit_wTrainTest.copy()

In [118]:
#### THIS CELL IS ONLY TO BE RUN IF YOURE IGNOREING THE TRAIN SPLIT THINGS AS YOU DID THEM BEFORE
#df_test_5 = df_all_Col_preSplit

In [119]:
def countRowsByClassOfNearPickOrNot(df,arrayOfClass,divisionInt,classToShrink):
    """
    Takes as input a dataframe, array of classes, an integer to divide by, and  a column, and a class within the column to shrink.
    Returns the dataframe minus the rows that match the ClassToShrink in the Col and prints details about the number of rows of the various classes.
    """
    for eachClass in arrayOfClass:
        print("length of rows with "+str(eachClass)+" in class_DistFrPick_TopTarget:",len(df[df['class_DistFrPick_TopTarget'] == eachClass]))
    df_NearPickZeroSmall = df.loc[(df.index%10 != 3) & (df['class_DistFrPick_TopTarget'] == classToShrink)]
    print("length of rows with 0 in class_DistFrPick_TopTarget and %"+str(divisionInt)+" == 0 is:",len(df_NearPickZeroSmall))
    print("% reduction in classs 0 is:", math.floor(len(df_NearPickZeroSmall) / len(df['class_DistFrPick_TopTarget'] == classToShrink) * 100),"%")
    total_after_reduction_in_bigger_class = len(df[df['class_DistFrPick_TopTarget'] == classToShrink]) -len(df_NearPickZeroSmall)
    print("if taken out using this remainder, the total number of 0 class will be: ",total_after_reduction_in_bigger_class)
#     print("ratio between that class away from pick and classes near pick is :":)
    return df_NearPickZeroSmall

In [120]:
class_array_NearPick = [100,95,70,60,0]
test_df_return = countRowsByClassOfNearPickOrNot(df_test_5,class_array_NearPick,2,0)

length of rows with 100 in class_DistFrPick_TopTarget: 973
length of rows with 95 in class_DistFrPick_TopTarget: 4049
length of rows with 70 in class_DistFrPick_TopTarget: 26066
length of rows with 60 in class_DistFrPick_TopTarget: 25654
length of rows with 0 in class_DistFrPick_TopTarget: 1245892
length of rows with 0 in class_DistFrPick_TopTarget and %2 == 0 is: 1121337
% reduction in classs 0 is: 86 %
if taken out using this remainder, the total number of 0 class will be:  124555


In [121]:
def dropsRowsWithMatchClassAndDeptRemainderIsZero(df,Col,RemainderInt,classToShrink):
    """
    Takes as input a dataframe, a column, a remainder integer, and a class within the column.
    Returns the dataframe minus the rows that match the ClassToShrink in the Col and have a depth from the DEPT col with a remainder of zero.
    """
    print("original lenght of dataframe = ",len(df))
    df_new = df.drop(df[(df[Col] == classToShrink) & (df.index%10 != 0)].index)
    print("length of new dataframe after dropping rows = ",len(df_new))
    print("number of rows dropped = ",len(df)-len(df_new))
    print("length of 0 class is :",len(df_new[df_new[Col] == classToShrink]))
    return df_new

In [122]:
df_all_Col_preSplit_wTrainTest_ClassBalanced = dropsRowsWithMatchClassAndDeptRemainderIsZero(df_all_Col_train_noRebalance,'class_DistFrPick_TopTarget',7,0)

original lenght of dataframe =  1046749
length of new dataframe after dropping rows =  145772
number of rows dropped =  900977
length of 0 class is : 100148


In [123]:
df_all_Col_preSplit_wTrainTest_ClassBalanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145772 entries, 0 to 1302630
Columns: 176 entries, CALI to diff_DEPT_vs_NN1_topTarget_DEPTH
dtypes: bool(2), float64(163), int64(8), object(3)
memory usage: 194.9+ MB


In [124]:
def addsRowsToBalanceClasses(df,rangeFor100,rangeFor95):
    """
    Input is a dataframe, range for class 100, and range for class 95
    Copies the rows with labels that don't occur very much so they are a larger part of dataframe
    returns the new dataframe with additional copies of rows added on
    """
    df_class100 = df[df['class_DistFrPick_TopTarget'] == 100]
    df_class95 = df[df['class_DistFrPick_TopTarget'] == 95]
    for each1 in range(rangeFor100):
        #print(each1)
        df = df.append(df_class100, ignore_index=True)
    for each2 in range(rangeFor95):
        #print(each2)
        df = df.append(df_class95, ignore_index=True)
    return df

In [125]:
df_all_Col_preSplit_wTrainTest_ClassBalanced2 = addsRowsToBalanceClasses(df_all_Col_preSplit_wTrainTest_ClassBalanced,50,10)

In [126]:
def findNumberOfEachClass(df,col):
    return df[col].value_counts()

In [127]:
findNumberOfEachClass(df_all_Col_preSplit_wTrainTest_ClassBalanced2,'class_DistFrPick_TopTarget')

0      100148
100     40086
95      35783
70      20947
60      20638
Name: class_DistFrPick_TopTarget, dtype: int64

In [128]:
len(df_all_Col_preSplit_wTrainTest_ClassBalanced2)

217602

In [129]:
df_all_Col_preSplit_wTrainTest_ClassBalanced2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217602 entries, 0 to 217601
Columns: 176 entries, CALI to diff_DEPT_vs_NN1_topTarget_DEPTH
dtypes: bool(2), float64(163), int64(8), object(3)
memory usage: 289.3+ MB


In [130]:
df_all_Col_preSplit_wTrainTest_ClassBalanced = df_all_Col_preSplit_wTrainTest_ClassBalanced2

# Identify which columns to use as features <a name="identifyTrainingFeatures"></a>

Get a list of columns

In [131]:
col_list = df_all_Col_preSplit_wTrainTest_ClassBalanced.columns
print(col_list)

Index(['CALI', 'COND', 'DELT', 'DEPT', 'DPHI', 'DT', 'GR', 'ILD', 'ILM', 'NPHI',
       ...
       'DPHI_min_11winSize_dirAroundnLarge', 'DPHI_min_21winSize_dirAroundMin', 'DPHI_min_21winSize_dirAboveMin', 'DPHI_min_21winSize_dirAroundMax', 'DPHI_min_21winSize_dirAboveMax', 'DPHI_min_21winSize_dirAroundMean', 'DPHI_min_21winSize_dirAboveMean', 'DPHI_min_21winSize_dirAbovenLarge', 'DPHI_min_21winSize_dirAroundnLarge', 'diff_DEPT_vs_NN1_topTarget_DEPTH'], dtype='object', length=176)


In [132]:
col_list = list(col_list)
col_list

['CALI',
 'COND',
 'DELT',
 'DEPT',
 'DPHI',
 'DT',
 'GR',
 'ILD',
 'ILM',
 'NPHI',
 'PHID',
 'RHOB',
 'SFL',
 'SFLU',
 'SN',
 'SP',
 'UWI',
 'trainOrTest',
 'SitID',
 'lat',
 'lng',
 'TopHelper_HorID',
 'TopTarget_HorID',
 'TopHelper_DEPTH',
 'TopTarget_DEPTH',
 'TopHelper_HorID_Qual',
 'TopTarget_Qual',
 'NN1_topTarget_DEPTH',
 'NN1_TopHelper_DEPTH',
 'NN1_thickness',
 'topTarget_Depth_predBy_NN1thick',
 'diff_Top_Depth_Real_v_predBy_NN1thick',
 'diff_TopTarget_DEPTH_v_rowDEPT',
 'diff_TopHelper_DEPTH_v_rowDEPT',
 'class_DistFrPick_TopTarget',
 'class_DistFrPick_TopHelper',
 'DistFrom_NN1ThickPredTopDepth_toRowDept',
 'NewWell',
 'LastBitWell',
 'TopWellDept',
 'BotWellDept',
 'FromTopWell',
 'FromBotWell',
 'WellThickness',
 'closerToBotOrTop',
 'closTopBotDist',
 'rowsToEdge',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirAboveMin',
 'GR_min_5winSize_dirAroundMax',
 'GR_min_5winSize_dirAboveMax',
 'GR_min_5winSize_dirAroundMean',
 'GR_min_5winSize_dirAboveMean',
 'GR_min_5w

## Manually copy the list above and take out some that are labels or aren't things you want to use as training
- At some point come back and see if I can instead use a standard list of things to not include and make the list of columns to use as features more automatically???

In [133]:
## NOTE WE ARE LEAVING THE UWI in for now but will take it out after dataframe is split into train/test portions!!!!
#train_feat_bigList = []

In [134]:
#len(train_feat_bigList)

In [135]:
#df_train_feat = df_all_Col_preSplit_wTrainTest_ClassBalanced[train_feat_bigList]

In [136]:
#df_train_feat.info()

Describing the dataframe here to find out which columns are not populated very much and have a lot of blanks. We'll likely exclude those columns. At this point doing it manually.

In [137]:
#df_train_feat.describe()

### Two lists of columns to not use as training features

Columns taken out as they aren't present often enough in the well dataset

In [138]:
training_feats_w_lowCount = ['RHOB','SP','CALI','COND','DELT','DENS','DPHI:1','DPHI:2','DT','GR:1','GR:2','IL','ILD:1','ILD:2','ILM','LITH','LLD','LLS','PHID','PHIN','RESD','RT','SFL','SFLU','SN','SNP','Sp']


Columns taken out as they either contain information probably captures in other columns, are related to labels too closely, or other reasons.
#### BUT LEAVE IN THE 'class_DistFrPick_TopTarget' column for now as thats a label we'll use in label df!!!!

In [263]:
takeOutColumnsNotCurvesList = [
    'FromBotWell',
    'FromTopWel'
    'rowsToEdge',
     'lat',
     'lng',  
 'SitID',
 'TopHelper_HorID',
 'TopTarget_HorID',
 'TopHelper_DEPTH',
 'diff_Top_Depth_Real_v_predBy_NN1thick',
 'diff_TopTarget_DEPTH_v_rowDEPT',
 'diff_TopHelper_DEPTH_v_rowDEPT',
 'class_DistFrPick_TopHelper',
 'NewWell',
 'LastBitWell',
 'TopWellDept',
 'BotWellDept',
 'WellThickness',
    'rowsToEdge',
    'closTopBotDist',
    'closerToBotOrTop'
]

Next few lines to combine the two lists above and take those columns out of dataframe

In [264]:
def takeOutColNotNeededInTrainingDF(df,list_allCol,colToTakeOutCurves,colToTakeOutOther):
    print("number of columns in dataframe coming into function",len(df.columns))
    train_feats_minusLowCount = [x for x in list_allCol if x not in colToTakeOutCurves]
    train_feats_minusLowCount = [x for x in train_feats_minusLowCount if x not in colToTakeOutOther]
    df_train_featWithHighCount = df[train_feats_minusLowCount]
    print("number of columns in dataframe leaving function",len(df_train_featWithHighCount.columns))
    return df_train_featWithHighCount

In [265]:
df_train_featWithHighCount = takeOutColNotNeededInTrainingDF(df_all_Col_preSplit_wTrainTest_ClassBalanced,col_list,training_feats_w_lowCount,takeOutColumnsNotCurvesList)


number of columns in dataframe coming into function 176
number of columns in dataframe leaving function 146


In [266]:
list(df_train_featWithHighCount.columns)

['DEPT',
 'DPHI',
 'GR',
 'ILD',
 'NPHI',
 'UWI',
 'trainOrTest',
 'TopTarget_DEPTH',
 'TopHelper_HorID_Qual',
 'TopTarget_Qual',
 'NN1_topTarget_DEPTH',
 'NN1_TopHelper_DEPTH',
 'NN1_thickness',
 'topTarget_Depth_predBy_NN1thick',
 'class_DistFrPick_TopTarget',
 'DistFrom_NN1ThickPredTopDepth_toRowDept',
 'FromTopWell',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirAboveMin',
 'GR_min_5winSize_dirAroundMax',
 'GR_min_5winSize_dirAboveMax',
 'GR_min_5winSize_dirAroundMean',
 'GR_min_5winSize_dirAboveMean',
 'GR_min_5winSize_dirAbovenLarge',
 'GR_min_5winSize_dirAroundnLarge',
 'GR_min_7winSize_dirAroundMin',
 'GR_min_7winSize_dirAboveMin',
 'GR_min_7winSize_dirAroundMax',
 'GR_min_7winSize_dirAboveMax',
 'GR_min_7winSize_dirAroundMean',
 'GR_min_7winSize_dirAboveMean',
 'GR_min_7winSize_dirAbovenLarge',
 'GR_min_7winSize_dirAroundnLarge',
 'GR_min_11winSize_dirAroundMin',
 'GR_min_11winSize_dirAboveMin',
 'GR_min_11winSize_dirAroundMax',
 'GR_min_11winSize_dirAboveMax',
 'GR_mi

Number of columns for training

In [267]:
len(df_train_featWithHighCount.columns)

146

In [268]:
df_train_featWithHighCount.describe()

,DEPT,DPHI,GR,ILD,NPHI,TopTarget_DEPTH,TopHelper_HorID_Qual,TopTarget_Qual,NN1_topTarget_DEPTH,NN1_TopHelper_DEPTH,NN1_thickness,topTarget_Depth_predBy_NN1thick,class_DistFrPick_TopTarget,DistFrom_NN1ThickPredTopDepth_toRowDept,FromTopWell,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,NPHI_min_5winSize_dirAroundMin,NPHI_min_5winSize_dirAboveMin,NPHI_min_5winSize_dirAroundMax,NPHI_min_5winSize_dirAboveMax,NPHI_min_5winSize_dirAroundMean,NPHI_min_5winSize_dirAboveMean,NPHI_min_5winSize_dirAbovenLarge,NPHI_min_5winSize_dirAroundnLarge,NPHI_min_7winSize_dirAroundMin,NPHI_min_7winSize_dirAboveMin,NPHI_min_7winSize_dirAroundMax,NPHI_min_7winSize_dirAboveMax,NPHI_min_7winSize_dirAroundMean,NPHI_min_7winSize_dirAboveMean,NPHI_min_7winSize_dirAbovenLarge,NPHI_min_7winSize_dirAroundnLarge,NPHI_min_11winSize_dirAroundMin,NPHI_min_11winSize_dirAboveMin,NPHI_min_11winSize_dirAroundMax,NPHI_min_11winSize_dirAboveMax,NPHI_min_11winSize_dirAroundMean,NPHI_min_11winSize_dirAboveMean,NPHI_min_11winSize_dirAbovenLarge,NPHI_min_11winSize_dirAroundnLarge,NPHI_min_21winSize_dirAroundMin,NPHI_min_21winSize_dirAboveMin,NPHI_min_21winSize_dirAroundMax,NPHI_min_21winSize_dirAboveMax,NPHI_min_21winSize_dirAroundMean,NPHI_min_21winSize_dirAboveMean,NPHI_min_21winSize_dirAbovenLarge,NPHI_min_21winSize_dirAroundnLarge,DPHI_min_5winSize_dirAroundMin,DPHI_min_5winSize_dirAboveMin,DPHI_min_5winSize_dirAroundMax,DPHI_min_5winSize_dirAboveMax,DPHI_min_5winSize_dirAroundMean,DPHI_min_5winSize_dirAboveMean,DPHI_min_5winSize_dirAbovenLarge,DPHI_min_5winSize_dirAroundnLarge,DPHI_min_7winSize_dirAroundMin,DPHI_min_7winSize_dirAboveMin,DPHI_min_7winSize_dirAroundMax,DPHI_min_7winSize_dirAboveMax,DPHI_min_7winSize_dirAroundMean,DPHI_min_7winSize_dirAboveMean,DPHI_min_7winSize_dirAbovenLarge,DPHI_min_7winSize_dirAroundnLarge,DPHI_min_11winSize_dirAroundMin,DPHI_min_11winSize_dirAboveMin,DPHI_min_11winSize_dirAroundMax,DPHI_min_11winSize_dirAboveMax,DPHI_min_11winSize_dirAroundMean,DPHI_min_11winSize_dirAboveMean,DPHI_min_11winSize_dirAbovenLarge,DPHI_min_11winSize_dirAroundnLarge,DPHI_min_21winSize_d

In [269]:
used_features = list(df_train_featWithHighCount.columns)

In [270]:
used_features

['DEPT',
 'DPHI',
 'GR',
 'ILD',
 'NPHI',
 'UWI',
 'trainOrTest',
 'TopTarget_DEPTH',
 'TopHelper_HorID_Qual',
 'TopTarget_Qual',
 'NN1_topTarget_DEPTH',
 'NN1_TopHelper_DEPTH',
 'NN1_thickness',
 'topTarget_Depth_predBy_NN1thick',
 'class_DistFrPick_TopTarget',
 'DistFrom_NN1ThickPredTopDepth_toRowDept',
 'FromTopWell',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirAboveMin',
 'GR_min_5winSize_dirAroundMax',
 'GR_min_5winSize_dirAboveMax',
 'GR_min_5winSize_dirAroundMean',
 'GR_min_5winSize_dirAboveMean',
 'GR_min_5winSize_dirAbovenLarge',
 'GR_min_5winSize_dirAroundnLarge',
 'GR_min_7winSize_dirAroundMin',
 'GR_min_7winSize_dirAboveMin',
 'GR_min_7winSize_dirAroundMax',
 'GR_min_7winSize_dirAboveMax',
 'GR_min_7winSize_dirAroundMean',
 'GR_min_7winSize_dirAboveMean',
 'GR_min_7winSize_dirAbovenLarge',
 'GR_min_7winSize_dirAroundnLarge',
 'GR_min_11winSize_dirAroundMin',
 'GR_min_11winSize_dirAboveMin',
 'GR_min_11winSize_dirAroundMax',
 'GR_min_11winSize_dirAboveMax',
 'GR_mi

### Now let's take out those same columns in the test only dataframe

In [271]:
df_test_featWithHighCount = takeOutColNotNeededInTrainingDF(df_all_Col_test,col_list,training_feats_w_lowCount,takeOutColumnsNotCurvesList)


number of columns in dataframe coming into function 176
number of columns in dataframe leaving function 146


--------------------

## Now let's combine the rebalanced train df with the unrebalanced test df to make a df we will then split into 4 pieces: train-data, train-labels, test-data,test-lables

In [272]:

df_testPlusRebalTrain_featWithHighCount = pd.concat([df_train_featWithHighCount,df_test_featWithHighCount])

In [273]:
len(df_testPlusRebalTrain_featWithHighCount.columns)

146

-----------------

## Identify which columns to use as labels<a name="identifyLabelCol"></a>

#### The column 'cat_isTopMcMrNearby_known' is what we'll use as labels.
- 100 = exactly the Top McMurray Pick
- 95 if the distance between that depth and the Top McMurray Pick is -0.5 < x and x <0.5
- 60 if the distance between that depth and the Top McMurray Pick is -5 < x and x < 5
- 0 = not near the Top McMurray Pick

The function used to make these classes or lables as column was:
`df_all_wells_wKNN_DEPTHtoDEPT['cat_isTopMcMrNearby_known']=df_all_wells_wKNN_DEPTHtoDEPT['diff_TMcM_Pick_v_DEPT'].apply(lambda x: 100 if x==0 else ( 95 if (-0.5 < x and x <0.5) else 60 if (-5 < x and x <5) else 0))`

In [274]:
df_testPlusRebalTrain_featWithHighCount['class_DistFrPick_TopTarget'].unique()

array([  0,  70,  95,  60, 100])

In [275]:
labels = df_testPlusRebalTrain_featWithHighCount[['class_DistFrPick_TopTarget','UWI','trainOrTest','TopTarget_DEPTH']]

In [276]:
labels.head()

,class_DistFrPick_TopTarget,UWI,trainOrTest,TopTarget_DEPTH
0,0,00/10-32-080-20W4/0,train,377.95
1,0,00/10-32-080-20W4/0,train,377.95
2,0,00/10-32-080-20W4/0,train,377.95
3,0,00/10-32-080-20W4/0,train,377.95
4,0,00/10-32-080-20W4/0,train,377.95


In [277]:
labels.tail()

,class_DistFrPick_TopTarget,UWI,trainOrTest,TopTarget_DEPTH
1296773,0,00/10-20-083-20W4/0,test,410.87
1296774,0,00/10-20-083-20W4/0,test,410.87
1296775,0,00/10-20-083-20W4/0,test,410.87
1296776,0,00/10-20-083-20W4/0,test,410.87
1296777,0,00/10-20-083-20W4/0,test,410.87


In [278]:
len(labels)

473487

The lengths of training dataframes and labels dataframes should be the same. We'll take out UWI and trainOrTest further down.

-----------------

## Now separate into 4 dataframes = <a name="splitDataframe"></a>
### train_labels
### train_feat 
### test_labels
### test_feat
Then take off UWI and TrainTest col

### Create label dataframes

In [279]:
#### split based on train in trainOrTest col
labels_train = labels[labels['trainOrTest'] == 'train' ]
#### Keep only the 'cat_isTopMcMrNearby_known' column, so now it is just a series of labels
labels_train = labels_train['class_DistFrPick_TopTarget']
#### split based on test in trainOrTest col
labels_test = labels[labels['trainOrTest'] == 'test' ]
#### Keep only the 'cat_isTopMcMrNearby_known' column, so now it is just a series of labels
labels_test = labels_test['class_DistFrPick_TopTarget']

In [200]:
df_train_featWithHighCount = df_testPlusRebalTrain_featWithHighCount

### Create training dataframes

In [280]:
#### split based on train in trainOrTest col and drop UWI and TrainOrTest columns
df_train_featWithHighCount_train = df_testPlusRebalTrain_featWithHighCount[df_testPlusRebalTrain_featWithHighCount['trainOrTest'] == 'train' ].drop(columns=['UWI', 'trainOrTest','class_DistFrPick_TopTarget','TopTarget_DEPTH'])
#### split based on test in trainOrTest col and drop UWI and TrainOrTest columns
df_train_featWithHighCount_test = df_testPlusRebalTrain_featWithHighCount[df_testPlusRebalTrain_featWithHighCount['trainOrTest'] == 'test' ].drop(columns=['UWI', 'trainOrTest','class_DistFrPick_TopTarget','TopTarget_DEPTH'])

## Create index dataframes for reattaching'UWI', 'trainOrTest','class_DistFrPick_TopTarget','TopTarget_DEPTH'

In [283]:
df_train_featWithHighCount_train_indexOnly = df_testPlusRebalTrain_featWithHighCount[df_testPlusRebalTrain_featWithHighCount['trainOrTest'] == 'train'][['UWI', 'trainOrTest','class_DistFrPick_TopTarget','TopTarget_DEPTH']]
df_train_featWithHighCount_test_indexOnly = df_testPlusRebalTrain_featWithHighCount[df_testPlusRebalTrain_featWithHighCount['trainOrTest'] == 'test' ][['UWI', 'trainOrTest','class_DistFrPick_TopTarget','TopTarget_DEPTH']]

In [284]:
df_train_featWithHighCount_train_indexOnly.head()

,UWI,trainOrTest,class_DistFrPick_TopTarget,TopTarget_DEPTH
0,00/10-32-080-20W4/0,train,0,377.95
1,00/10-32-080-20W4/0,train,0,377.95
2,00/10-32-080-20W4/0,train,0,377.95
3,00/10-32-080-20W4/0,train,0,377.95
4,00/10-32-080-20W4/0,train,0,377.95


### Rename to avoid overwriting & keep with previous work

In [285]:
train_X = df_train_featWithHighCount_train
train_y = labels_train
test_X = df_train_featWithHighCount_test
test_y = labels_test
train_index = df_train_featWithHighCount_train_indexOnly
test_index = df_train_featWithHighCount_test_indexOnly 

### Inspect to make sure column headers and lengths make sense

In [298]:
print(len(train_X))
train_X.head()

217602


,DEPT,DPHI,GR,ILD,NPHI,TopHelper_HorID_Qual,TopTarget_Qual,NN1_topTarget_DEPTH,NN1_TopHelper_DEPTH,NN1_thickness,topTarget_Depth_predBy_NN1thick,DistFrom_NN1ThickPredTopDepth_toRowDept,FromTopWell,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,NPHI_min_5winSize_dirAroundMin,NPHI_min_5winSize_dirAboveMin,NPHI_min_5winSize_dirAroundMax,NPHI_min_5winSize_dirAboveMax,NPHI_min_5winSize_dirAroundMean,NPHI_min_5winSize_dirAboveMean,NPHI_min_5winSize_dirAbovenLarge,NPHI_min_5winSize_dirAroundnLarge,NPHI_min_7winSize_dirAroundMin,NPHI_min_7winSize_dirAboveMin,NPHI_min_7winSize_dirAroundMax,NPHI_min_7winSize_dirAboveMax,NPHI_min_7winSize_dirAroundMean,NPHI_min_7winSize_dirAboveMean,NPHI_min_7winSize_dirAbovenLarge,NPHI_min_7winSize_dirAroundnLarge,NPHI_min_11winSize_dirAroundMin,NPHI_min_11winSize_dirAboveMin,NPHI_min_11winSize_dirAroundMax,NPHI_min_11winSize_dirAboveMax,NPHI_min_11winSize_dirAroundMean,NPHI_min_11winSize_dirAboveMean,NPHI_min_11winSize_dirAbovenLarge,NPHI_min_11winSize_dirAroundnLarge,NPHI_min_21winSize_dirAroundMin,NPHI_min_21winSize_dirAboveMin,NPHI_min_21winSize_dirAroundMax,NPHI_min_21winSize_dirAboveMax,NPHI_min_21winSize_dirAroundMean,NPHI_min_21winSize_dirAboveMean,NPHI_min_21winSize_dirAbovenLarge,NPHI_min_21winSize_dirAroundnLarge,DPHI_min_5winSize_dirAroundMin,DPHI_min_5winSize_dirAboveMin,DPHI_min_5winSize_dirAroundMax,DPHI_min_5winSize_dirAboveMax,DPHI_min_5winSize_dirAroundMean,DPHI_min_5winSize_dirAboveMean,DPHI_min_5winSize_dirAbovenLarge,DPHI_min_5winSize_dirAroundnLarge,DPHI_min_7winSize_dirAroundMin,DPHI_min_7winSize_dirAboveMin,DPHI_min_7winSize_dirAroundMax,DPHI_min_7winSize_dirAboveMax,DPHI_min_7winSize_dirAroundMean,DPHI_min_7winSize_dirAboveMean,DPHI_min_7winSize_dirAbovenLarge,DPHI_min_7winSize_dirAroundnLarge,DPHI_min_11winSize_dirAroundMin,DPHI_min_11winSize_dirAboveMin,DPHI_min_11winSize_dirAroundMax,DPHI_min_11winSize_dirAboveMax,DPHI_min_11winSize_dirAroundMean,DPHI_min_11winSize_dirAboveMean,DPHI_min_11winSize_dirAbovenLarge,DPHI_min_11winSize_dirAroundnLarge,DPHI_min_21winSize_dirAroundMin,DPHI_min_21winSize_dirAboveMin,

In [299]:
print(len(train_y))
train_y.head()

217602


0    0
1    0
2    0
3    0
4    0
Name: class_DistFrPick_TopTarget, dtype: int64

In [300]:
print(len(test_X))
test_X.head()

255885


,DEPT,DPHI,GR,ILD,NPHI,TopHelper_HorID_Qual,TopTarget_Qual,NN1_topTarget_DEPTH,NN1_TopHelper_DEPTH,NN1_thickness,topTarget_Depth_predBy_NN1thick,DistFrom_NN1ThickPredTopDepth_toRowDept,FromTopWell,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,NPHI_min_5winSize_dirAroundMin,NPHI_min_5winSize_dirAboveMin,NPHI_min_5winSize_dirAroundMax,NPHI_min_5winSize_dirAboveMax,NPHI_min_5winSize_dirAroundMean,NPHI_min_5winSize_dirAboveMean,NPHI_min_5winSize_dirAbovenLarge,NPHI_min_5winSize_dirAroundnLarge,NPHI_min_7winSize_dirAroundMin,NPHI_min_7winSize_dirAboveMin,NPHI_min_7winSize_dirAroundMax,NPHI_min_7winSize_dirAboveMax,NPHI_min_7winSize_dirAroundMean,NPHI_min_7winSize_dirAboveMean,NPHI_min_7winSize_dirAbovenLarge,NPHI_min_7winSize_dirAroundnLarge,NPHI_min_11winSize_dirAroundMin,NPHI_min_11winSize_dirAboveMin,NPHI_min_11winSize_dirAroundMax,NPHI_min_11winSize_dirAboveMax,NPHI_min_11winSize_dirAroundMean,NPHI_min_11winSize_dirAboveMean,NPHI_min_11winSize_dirAbovenLarge,NPHI_min_11winSize_dirAroundnLarge,NPHI_min_21winSize_dirAroundMin,NPHI_min_21winSize_dirAboveMin,NPHI_min_21winSize_dirAroundMax,NPHI_min_21winSize_dirAboveMax,NPHI_min_21winSize_dirAroundMean,NPHI_min_21winSize_dirAboveMean,NPHI_min_21winSize_dirAbovenLarge,NPHI_min_21winSize_dirAroundnLarge,DPHI_min_5winSize_dirAroundMin,DPHI_min_5winSize_dirAboveMin,DPHI_min_5winSize_dirAroundMax,DPHI_min_5winSize_dirAboveMax,DPHI_min_5winSize_dirAroundMean,DPHI_min_5winSize_dirAboveMean,DPHI_min_5winSize_dirAbovenLarge,DPHI_min_5winSize_dirAroundnLarge,DPHI_min_7winSize_dirAroundMin,DPHI_min_7winSize_dirAboveMin,DPHI_min_7winSize_dirAroundMax,DPHI_min_7winSize_dirAboveMax,DPHI_min_7winSize_dirAroundMean,DPHI_min_7winSize_dirAboveMean,DPHI_min_7winSize_dirAbovenLarge,DPHI_min_7winSize_dirAroundnLarge,DPHI_min_11winSize_dirAroundMin,DPHI_min_11winSize_dirAboveMin,DPHI_min_11winSize_dirAroundMax,DPHI_min_11winSize_dirAboveMax,DPHI_min_11winSize_dirAroundMean,DPHI_min_11winSize_dirAboveMean,DPHI_min_11winSize_dirAbovenLarge,DPHI_min_11winSize_dirAroundnLarge,DPHI_min_21winSize_dirAroundMin,DPHI_min_21winSize_dirAboveMin,

In [301]:
print(len(test_y))
test_y.head()

255885


11203    0
11204    0
11205    0
11206    0
11207    0
Name: class_DistFrPick_TopTarget, dtype: int64

-------------------

## Save the dataframes as a dict
- train_X 
- train_y 
- test_X
- test_y
- & full

### Write pandas dataframes to HDF5

In [302]:
# Write hdf5 to current directory
# df = df_all_Col_preSplit_wTrainTest_ClassBalanced
# key = preSplit
df_testPlusRebalTrain_featWithHighCount.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='preSplitpreBal', mode='w')

In [303]:
# Write hdf5 to current directory
# df = df_all_Col_preSplit_wTrainTest_ClassBalanced
# key = preSplit

train_X.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='train_X')

In [304]:
# Write hdf5 to current directory
# df = train_y
# key = train_y

train_y.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='train_y')

In [305]:
# Write hdf5 to current directory
# df = df_all_Col_preSplit_wTrainTest_ClassBalanced
# key = test_X

test_X.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='test_X')

In [310]:
# Write hdf5 to current directory
# df = df_all_Col_preSplit_wTrainTest_ClassBalanced
# key = test_y

test_y.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='test_y')

In [307]:
# train_index.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='test_y')

In [308]:
train_index.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='train_index')

In [309]:
test_index.to_hdf('df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML_20181003.h5', key='test_index')

---------------------

# Machine-learning<a name=machineLearningNoDask></a>

In [244]:
seed = 123

In [245]:
# .values.ravel()
model = XGBClassifier(
    gamma=0, 
    reg_alpha=0.2, 
    max_depth=3, 
    subsample=0.8, 
    colsample_bytree= 0.8, 
    n_estimators= 300, 
    learning_rate= 0.03, 
    min_child_weight= 3,n_jobs=8)
model.fit(train_X,train_y)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=3, missing=None, n_estimators=300,
       n_jobs=8, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0.2, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [246]:
result = model.predict(test_X)
result

/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0, 0, 0, ..., 0, 0, 0])

In [247]:
type(result)

numpy.ndarray

In [248]:
len(result)

255885

In [249]:
test_y[3300:4900]

31605      0
31606      0
31607      0
31608      0
31609      0
31610      0
31611      0
31612      0
31613      0
31614      0
31615      0
31616      0
31617      0
31618      0
31619      0
31620      0
31621      0
31622      0
31623      0
31624      0
31625      0
31626      0
31627      0
31628      0
31629      0
31630      0
31631      0
31632      0
31633      0
31634      0
31635      0
31636      0
31637      0
31638      0
31639      0
31640      0
31641      0
31642      0
31643      0
31644      0
31645      0
31646      0
31647      0
31648      0
31649      0
31650      0
31651      0
31652      0
31653      0
31654      0
31655      0
31656      0
31657      0
31658      0
31659      0
31660      0
31661      0
31662      0
31663      0
31664      0
31665      0
31666      0
31667      0
31668      0
31669      0
31670      0
31671      0
31672      0
31673      0
31674      0
31675      0
31676      0
31677      0
31678      0
31679      0
31680      0
31681      0

In [250]:
test_y_indexValues = test_y.index.values
df_result = pd.DataFrame(result, index=test_y_indexValues, columns=['TopTarget_Pick_pred'])
df_results_2 = pd.concat([test_y, df_result], axis=1)

In [251]:
df_results_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255885 entries, 11203 to 1296777
Data columns (total 2 columns):
class_DistFrPick_TopTarget    255885 non-null int64
TopTarget_Pick_pred           255885 non-null int64
dtypes: int64(2)
memory usage: 5.9 MB


In [252]:
df_results_2.head()

,class_DistFrPick_TopTarget,TopTarget_Pick_pred
11203,0,0
11204,0,0
11205,0,0
11206,0,0
11207,0,0


In [253]:
# test_df_pred = test_y.copy()
# test_df_pred['Pick_pred'] = result
# test_df_pred.head()

# Examination of first-level results<a name="ml_evaluation"></a>

In [254]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# make predictions for test data
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(df_results_2['class_DistFrPick_TopTarget'], df_results_2['TopTarget_Pick_pred'])

#### Results of accuracy prediction where only exact label matches count on a row by row basis, so 60=60,100=100

In [255]:
accuracy

0.8886296578541142

#### Making another dataframe to make rows that lump in classes to combare to other groups of prediction classes

In [257]:
df_results_3 = df_results_2.copy()

In [258]:
df_results_3[0:500]

,class_DistFrPick_TopTarget,TopTarget_Pick_pred
11203,0,0
11204,0,0
11205,0,0
11206,0,0
11207,0,0
11208,0,0
11209,0,0
11210,0,0
11211,0,0
11212,0,0


In [259]:
df_results_3['class_DistFrPick_TopTarget_95or100'] = np.where(df_results_3['class_DistFrPick_TopTarget']>70, 1, 0)
df_results_3['TopTarget_Pick_pred_95or100'] = np.where(df_results_3['TopTarget_Pick_pred']>70, 1, 0)

In [261]:
#### inspect
df_results_3[500:800]

,class_DistFrPick_TopTarget,TopTarget_Pick_pred,class_DistFrPick_TopTarget_95or100,TopTarget_Pick_pred_95or100
11703,0,0,0,0
11704,0,0,0,0
11705,0,0,0,0
11706,0,0,0,0
11707,0,0,0,0
11708,0,0,0,0
11709,0,0,0,0
11710,0,0,0,0
11711,0,0,0,0
11712,0,0,0,0


#### accuracy if looking at only the labels for 95 and 100 in both known and prediction

In [262]:
accuracy = accuracy_score(df_results_3['class_DistFrPick_TopTarget'], df_results_3['TopTarget_Pick_pred_95or100'])
accuracy

0.9021669890771244

Create more columns for lumped labels

In [88]:
df_results_3['cat_isTopMcMrNearby_known_60or95or100'] = np.where(df_results_3['cat_isTopMcMrNearby_known']>59, 1, 0)
df_results_3['TopMcMr_Pick_pred_60or95or100'] = np.where(df_results_3['TopMcMr_Pick_pred']>59, 1, 0)
df_results_3['cat_isTopMcMrNearby_known_100'] = np.where(df_results_3['cat_isTopMcMrNearby_known']==100, 1, 0)
df_results_3['TopMcMr_Pick_pred_known_100'] = np.where(df_results_3['TopMcMr_Pick_pred']==100, 1, 0)

In [89]:
accuracy = accuracy_score(df_results_3['cat_isTopMcMrNearby_known_60or95or100'], df_results_3['TopMcMr_Pick_pred_60or95or100'])
accuracy

0.87035775680321759

In [90]:
accuracy = accuracy_score(df_results_3['cat_isTopMcMrNearby_known_100'], df_results_3['TopMcMr_Pick_pred_60or95or100'])
accuracy

0.62803671629204372

In [91]:
#### inspecting results manually
df_results_3[7000:9000]

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred,cat_isTopMcMrNearby_known_95or100,TopMcMr_Pick_pred_95or100,cat_isTopMcMrNearby_known_60or95or100,TopMcMr_Pick_pred_60or95or100,cat_isTopMcMrNearby_known_100,TopMcMr_Pick_pred_known_100
39398,60,100,0,1,1,1,0,1
39399,60,100,0,1,1,1,0,1
39400,60,100,0,1,1,1,0,1
39401,0,60,0,0,0,1,0,0
39402,0,0,0,0,0,0,0,0
39403,0,0,0,0,0,0,0,0
39404,0,0,0,0,0,0,0,0
39405,0,0,0,0,0,0,0,0
39406,0,0,0,0,0,0,0,0
39407,0,0,0,0,0,0,0,0


In [92]:
len(df_results_3)

61662

In [93]:
df_all_Col_preSplit_wTrainTest_ClassBalanced.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.0,246.5,246.5,FromTopWell,0.0,0,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,train
1,166.675,NaN,NaN,NaN,152.102,NaN,0.269,NaN,NaN,NaN,26.625,NaN,NaN,NaN,30.179,NaN,NaN,NaN,NaN,NaN,NaN,0.355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,225.848,232.558,0,0,207.558,False,False,149.602,396.102,2.5,244.0,246.5,FromTopWell,2.5,10,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,2

In [94]:
len(df_all_Col_preSplit_wTrainTest_ClassBalanced)

307648

In [95]:
df_all_Col_preSplit_wTrainTest_ClassBalanced_Copy = np.where(df_all_Col_preSplit_wTrainTest_ClassBalanced['trainOrTest'] == 'test')

In [96]:
len(df_all_Col_preSplit_wTrainTest_ClassBalanced_Copy)

1

In [98]:
predictedPickIsExactlyHere = df_results_3[df_results_3['TopMcMr_Pick_pred_known_100'] == 1]
test100 = predictedPickIsExactlyHere['TopMcMr_Pick_pred_known_100']

In [99]:
type(test100)

pandas.core.series.Series

In [100]:
test100.values

array([1, 1, 1, ..., 1, 1, 1])

### More evaluation

In [102]:
df_featPlus_wUWI_testCopy = df_train_featWithHighCount[df_train_featWithHighCount['trainOrTest'] == 'test' ].copy()

In [103]:
df_featPlus_wUWI_testCopy.head()

,UWI,trainOrTest,DPHI,GR,ILD,NPHI,McMurray_Base_Qual,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
390,00/11-19-073-16W4/0,test,0.185,101.752,3.723,0.537,2,23.78,421.84,1,2,183.096,445.994,1.0,208.25,101.752,101.752,101.752,101.752,101.7520,101.7520,101.7520,101.7520,101.752,101.752,101.752,101.752,101.752000,101.752000,101.752,101.7520,101.752,101.752,101.752,101.752,101.752000,101.752,101.752,101.752,101.752,101.752,101.752,101.752,101.75200,101.752,101.752,101.752,3.723,3.723,3.723,3.723,3.7230,3.7230,3.7230,3.7230,3.723,3.723,3.723,3.723,3.723000,3.723000,3.7230,3.7230,3.723,3.723,3.723,3.723,3.723000,3.723,3.723,3.723,3.723,3.723,3.723,3.723,3.723000,3.723,3.723,3.723
391,00/11-19-073-16W4/0,test,0.212,100.657,2.950,0.516,2,23.78,421.84,1,2,180.596,445.994,3.5,208.25,100.349,100.657,104.476,100.657,101.5134,100.6570,100.6570,100.6570,100.349,100.657,106.802,100.657,102.304429,100.657000,100.657,100.6570,100.657,100.657,100.657,100.657,100.657000,100.657,100.657,100.657,100.657,100.657,100.657,100.657,100.65700,100.657,100.657,100.657,2.950,2.950,3.254,2.950,3.1066,2.9500,2.9500,2.9500,2.950,2.950,3.414,2.950,3.194286,2.950000,2.9500,2.9500,2.950,2.950,2.950,2.950,2.950000,2.950,2.950,2.950,2.950,2.950,2.950,2.950,2.950000,2.950,2.950,2.950
392,00/11-19-073-16W4/0,test,0.175,100.744,3.409,0.532,2,23.78,421.84,1,2,178.096,445.994,6.0,208.25,99.221,100.744,106.397,106.397,102.5820,104.5656,104.5656,102.5820,99.221,100.744,106.397,100.744,103.294000,100.744000,100.744,100.7440,99.221,100.744,106.729,100.744,103.907273,100.744,100.744,100.744,100.744,100.744,100.744,100.744,100.74400,100.744,100.744,100.744,3.299,3.299,3.558,3.493,3.4230,3.3906,3.3906,3.4230,3.299,3.409,3.632,3.409,3.449143,3.409000,3.4090,3.4090,3.299,3.409,3.632,3.409,3.478455,3.409,3.409,3.409,3.409,3.409,3.409,3.409,3.409000,3.409,3.409,3.409
393,00/11-19-073-16W4/0,test,0.265,91.018,4.864,0.489,2,23.78,421.84,1,2,175.596,445.994,8.5,208.25,67.810,91.018,102.6

In [115]:
len(df_featPlus_wUWI_testCopy)

61662

In [104]:
df_featPlus_wUWI_testCopy_wResults = pd.concat([df_featPlus_wUWI_testCopy, df_results_3], axis=1)
df_featPlus_wUWI_testCopy_wResults.tail()

,UWI,trainOrTest,DPHI,GR,ILD,NPHI,McMurray_Base_Qual,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred,cat_isTopMcMrNearby_known_95or100,TopMcMr_Pick_pred_95or100,cat_isTopMcMrNearby_known_60or95or100,TopMcMr_Pick_pred_60or95or100,cat_isTopMcMrNearby_known_100,TopMcMr_Pick_pred_known_100
307633,00/16-29-073-05W5/0,test,0.132,69.814,7.565,0.377,1,3.00,612.00,1,1,20.250,595.000,231.75,235.00,58.985,69.814,79.901,69.814,69.5914,69.8140,69.8140,69.8140,58.985,69.814,80.779,69.814,70.854714,69.814000,69.8140,69.8140,69.814,69.814,69.814,69.814,69.814000,69.814000,69.8140,69.8140,69.814,69.814,69.814,69.814,69.814000,69.814,69.814,69.814,4.601,7.565,9.848,7.565,7.3860,7.5650,7.5650,7.5650,3.528,7.565,9.848,7.565,7.104429,7.565000,7.5650,7.5650,7.565,7.565,7.565,7.565,7.565000,7.565000,7.5650,7.5650,7.565,7.565,7.565,7.565,7.565000,7.565,7.565,7.565,95,0,1,0,1,0,0,0
307634,00/06-26-075-21W4/0,test,0.234,52.644,18.299,0.312,3,17.07,562.66,3,3,13.032,594.442,182.50,201.25,39.106,39.106,94.659,101.032,62.0152,65.0900,65.0900,62.0152,39.106,39.106,94.659,124.887,69.691286,80.616286,95.3720,80.0770,39.106,39.106,113.977,146.483,77.909727,101.844273,136.1720,97.4862,39.106,52.644,146.483,52.644,93.937857,52.644,52.644,52.644,16.342,13.736,18.642,18.642,17.5814,16.5280,16.5280,17.5814,14.874,10.919,18.642,18.642,16.854857,15.112000,16.5280,17.5814,12.225,7.541,18.642,18.642,15.585273,12.802636,16.5280,17.5814,7.541,18.299,18.642,18.299,13.319762,18.299,18.299,18.299,95,60,1,0,1,1,0,0
307635,00/06-26-075-21W4/0,test,0.211,75.319,17.535,0.336,3,17.07,562.66,3,3,13.282,594.442,182.75,201.25,39.106,39.106,94.659,84.320,71.0342,59.9474,59.9474,71.0342,39.106,39.106,94.659,113.977,69.303857,73.535143,85.4584,79.5346,39.106,39.106,101.032,142.116,74.123182,95.374818,129.6708,91.0124,39.106,75.319,142.116,75.319,91.498381,75.319,75.319,75.319,15.203,14.874,18.642,18.642,17.2042,17.2878,17.2878,17.2042,14.116,12.2

In [116]:
len(df_featPlus_wUWI_testCopy_wResults)

61662

In [105]:
wells_in_test = df_featPlus_wUWI_testCopy_wResults['UWI'].unique()
len(wells_in_test)

382

limt new dataframe to rows that are less than 1 from actual pick

In [107]:
df_look_at_pred_class_vs_distFromRealLess1 = df_featPlus_wUWI_testCopy_wResults[df_featPlus_wUWI_testCopy_wResults['DistFrom_NN1_TopDepth_Abs'] < 1 ]

In [108]:
df_look_at_pred_class_vs_distFromRealLess1['cat_isTopMcMrNearby_known'].nunique()

4

groupy label and get counts as dataframe using nunique

In [118]:
df_count = df_look_at_pred_class_vs_distFromRealLess1.groupby('TopMcMr_Pick_pred').nunique()

In [119]:
df_count

,UWI,trainOrTest,DPHI,GR,ILD,NPHI,McMurray_Base_Qual,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred,cat_isTopMcMrNearby_known_95or100,TopMcMr_Pick_pred_95or100,cat_isTopMcMrNearby_known_60or95or100,TopMcMr_Pick_pred_60or95or100,cat_isTopMcMrNearby_known_100,TopMcMr_Pick_pred_known_100
TopMcMr_Pick_pred,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,17,1,21,22,22,22,3,17,17,3,3,12,17,21,14,18,19,20,22,22,22,22,22,17,20,21,20,22,22,22,22,19,20,20,22,22,22,22,21,20,20,22,20,22,22,21,22,20,17,22,22,22,22,22,22,18,17,22,22,22,22,22,22,17,22,22,22,22,22,22,22,22,22,21,22,22,22,22,22,2,1,1,1,2,1,1,1
60,212,1,260,620,610,247,4,163,202,3,4,300,177,409,135,446,466,492,480,623,623,623,622,413,444,472,439,623,623,596,600,371,399,427,393,623,623,531,567,340,404,371,371,623,622,462,490,543,536,497,522,617,616,614,611,519,487,450,498,617,616,593,564,470,442,398,474,615,616,560,501,418,398,383,467,616,615,532,464,4,1,2,1,2,1,2,1
95,119,1,152,287,269,150,4,104,115,3,4,187,106,257,87,248,247,222,232,287,287,287,287,239,217,194,225,287,287,279,259,217,193,169,225,287,287,269,223,187,199,153,216,287,287,255,210,217,224,254,242,274,274,273,273,201,223,245,207,274,273,253,271,174,224,215,189,274,274,216,260,171,209,175,200,273,274,215,222,4,1,2,1,2,1,2,1
100,69,1,94,137,137,88,4,56,65,3,4,34,58,107,46,126,122,111,113,137,137,137,137,119,112,99,113,137,137,132,132,105,100,95,106,137,137,128,119,91,89,88,99,137,137,126,109,118,125,130,122,137,137,137,137,111,120,125,117,137,137,134,134,102,111,115,100,137,137,118,130,96,109,85,97,137,137,106,103,4,1,2,1,2,1,2,1


In [120]:
total_rows_less_than_1_from_pick = df_count['UWI'].unique().sum()
total_rows_less_than_1_from_pick

417

Why is the number of unique wells less than the number calculated above 382? Where there rows included in the test dataset that didn't have any rows within 1 of the pick for that well?

In [121]:
df_count['UWI']

TopMcMr_Pick_pred
0       17
60     212
95     119
100     69
Name: UWI, dtype: int64

In [132]:
def getPercents(df,total_wells):
    index_list = df.index.values
    index_num = -1
    for Each in df:
        index_num = index_num+1
        print("label is =", index_num," and total instaces of that label =",Each, "and the % is: ",Each/total_wells)

In [133]:
getPercents(df_count['UWI'],total_rows_less_than_1_from_pick)

label is = 0  and total instaces of that label = 17 and the % is:  0.0407673860911
label is = 1  and total instaces of that label = 212 and the % is:  0.508393285372
label is = 2  and total instaces of that label = 119 and the % is:  0.285371702638
label is = 3  and total instaces of that label = 69 and the % is:  0.165467625899


#### The numbers above show the number of rows within 1 of the actual pick in terms of their predicted label.
#### What we see from this is there are very few rows within 1 (foot?) of actual pick that are predicted to be class 0, or more than 5 from the pick. 

In [134]:
df_look_at_pred_class_vs_distFromRealLess1 = df_featPlus_wUWI_testCopy_wResults[df_featPlus_wUWI_testCopy_wResults['DistFrom_NN1_TopDepth_Abs'] < 1 ]

In [135]:
df_count = df_look_at_pred_class_vs_distFromRealLess1.groupby('TopMcMr_Pick_pred').nunique()

In [136]:
total_rows_less_than_1_from_pick = df_count['UWI'].unique().sum()
total_rows_less_than_1_from_pick

417

In [137]:
df_count['UWI']

TopMcMr_Pick_pred
0       17
60     212
95     119
100     69
Name: UWI, dtype: int64

In [138]:
getPercents(df_count['UWI'],total_rows_less_than_1_from_pick)

label is = 0  and total instaces of that label = 17 and the % is:  0.0407673860911
label is = 1  and total instaces of that label = 212 and the % is:  0.508393285372
label is = 2  and total instaces of that label = 119 and the % is:  0.285371702638
label is = 3  and total instaces of that label = 69 and the % is:  0.165467625899


In [139]:
def getStatsOnWithinDistOfPick(df,distOfPick):
    df_look_at_pred_class_vs_distFromRealLessNum = df[df['DistFrom_NN1_TopDepth_Abs'] < distOfPick]
    df_count = df_look_at_pred_class_vs_distFromRealLessNum.groupby('TopMcMr_Pick_pred').nunique()
    total_rows_less_than_Num_from_pick = df_count['UWI'].unique().sum()
    getPercents(df_count['UWI'],total_rows_less_than_Num_from_pick)

In [140]:
getStatsOnWithinDistOfPick(df_featPlus_wUWI_testCopy_wResults,5)

label is = 0  and total instaces of that label = 53 and the % is:  0.0706666666667
label is = 1  and total instaces of that label = 329 and the % is:  0.438666666667
label is = 2  and total instaces of that label = 198 and the % is:  0.264
label is = 3  and total instaces of that label = 170 and the % is:  0.226666666667


### What this tells us is most of the predicted classes at or around the pick are predicted class of at or around the pick, which is good!

----------

# Turning row-by-row classification into single pick value prediction<a name="classificationToPick"></a>

1. Create function that treats depth & classification prediction column like histogram and finds median value (in this case depth)
2. Create widgeted function that changes values of labels to shift how much weight is given to each class (at pick, right by pick, sorta nearby pick, etc.)
3. Visualize step #2
4. Use steps 1,2,3 to create a new prediction that is a depth for each well
5. Calculate average distance between actual pick and predicted pick
6. Plot results of step 5 as simple scatter plot
7. Plot results of step 5 as map